In [ ]:
# Q1. Notebook
# What's the standard deviation of the predicted duration for this dataset?
# A:6.247488852238703


# Q2. Preparing the output
# What's the size of the output file?
# A:66M

# Q3. Creating the scoring script
# Which command you need to execute for that?
# A: nbconvert 

# Q4. Virtual environment
# What's the first hash for the Scikit-Learn dependency?
# sha256:057b991ac64b3e75c9c04b5f9395eaf19a6179244c089afdebaad98264bff37c

# Q5. Parametrize the script
# What's the mean predicted duration?
# A: 14.29

# Q6. Docker container
# Now run the script with docker. What's the mean predicted duration for May 2023?
# 0.19174419265916945

In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.6.1


In [2]:
!python -V

Python 3.12.1


In [3]:
import pickle
import pandas as pd

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [8]:
!pip install pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 82.5 MB/s eta 0:00:00:00:01

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [9]:
year = 2023
month = 3
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [10]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [ ]:
# Q1. Notebook
# What's the standard deviation of the predicted duration for this dataset?
# A:6.247488852238703
import numpy as np 

stdev  = np.std(y_pred)

print(stdev)

6.247488852238703


In [17]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

df_result = df[['ride_id']]
df_result['duration_predicted'] = y_pred

print(df_result)

                 ride_id  duration_predicted
0              2023/03_0           16.245906
1              2023/03_1           26.134796
2              2023/03_2           11.884264
3              2023/03_3           11.997720
4              2023/03_4           10.234486
...                  ...                 ...
3403761  2023/03_3403761           11.952635
3403762  2023/03_3403762           20.049958
3403763  2023/03_3403763           11.595336
3403764  2023/03_3403764           13.113178
3403765  2023/03_3403765           12.899992

[3316216 rows x 2 columns]


/tmp/ipykernel_5890/1683946010.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_result['duration_predicted'] = y_pred


In [18]:
output_file = 'df_result.parquet'

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)